**DESAFIO ENACOM**

**OBJETIVO**: Desenvolver um modelo de previsão de geração de energia hidrelétrica anual.

In [1]:
# Importando as bibliotecas necessárias
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Obtendo os dados no DataBase

Os dados referentes a **Dados Hidrológicos de Reservatório** foram coletados do portal do Operador Nacional do Sistema Elétrico (ONS) e estão disponíveis no seguinte site: https://dados.ons.org.br/dataset/dados-hidrologicos-res

In [2]:
# Criar listas de anos e números
anos = list(range(2000, 2021)) # Para o desafio, deve-se considerar o range até 2021.

# Criar função para obter os dados hidrológicos de reservatório
def download_csv(ano):
    try:
      if ano <= 2021:
        url = f"https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/dados_hidrologicos_di/DADOS_HIDROLOGICOS_RES_{ano}.csv"
        # Imprimir a URL antes de fazer o download
        print(f"URL: {url}")
        return pd.read_csv(url, sep=';')
    except Exception as e:
        print(f"Erro ao baixar dados de {url}: {e}")
        return None

# Baixar os DataFrames com apenas no ano (correspodem os dados de 2000 a 2021)
dado_hidrologico = pd.concat([download_csv(ano) for ano in anos], ignore_index=True)

URL: https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/dados_hidrologicos_di/DADOS_HIDROLOGICOS_RES_2000.csv
URL: https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/dados_hidrologicos_di/DADOS_HIDROLOGICOS_RES_2001.csv
URL: https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/dados_hidrologicos_di/DADOS_HIDROLOGICOS_RES_2002.csv
URL: https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/dados_hidrologicos_di/DADOS_HIDROLOGICOS_RES_2003.csv
URL: https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/dados_hidrologicos_di/DADOS_HIDROLOGICOS_RES_2004.csv
URL: https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/dados_hidrologicos_di/DADOS_HIDROLOGICOS_RES_2005.csv
URL: https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/dados_hidrologicos_di/DADOS_HIDROLOGICOS_RES_2006.csv
URL: https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/dados_hidrologicos_di/DADOS_HIDROLOGICOS_RES_2007.csv
URL: https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/dados_hidrologicos_di/DADOS_

In [ ]:
# Visualizando o dataframe
dado_hidrologico.head()

,id_subsistema,din_instante,val_nivelmontante,val_niveljusante,val_volumeutilcon,val_vazaoafluente,val_vazaoturbinada,val_vazaovertida,val_vazaooutrasestruturas,val_vazaodefluente,val_vazaotransferida,val_vazaonatural,val_vazaoartificial,val_vazaoincremental,val_vazaoevaporacaoliquida,val_vazaousoconsuntivo
0,NE,2000-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,350.72,NaN,150.96,NaN,NaN
1,NE,2000-01-01,115.20,NaN,117.20,421.87,NaN,NaN,NaN,508.0,NaN,422.02,NaN,422.02,NaN,NaN
2,NE,2000-01-01,301.59,259.57,57.03,868.39,417.0,0.0,NaN,417.0,NaN,814.64,NaN,58.56,NaN,NaN
3,NE,2000-01-01,384.42,NaN,19.44,2810.00,NaN,0.0,NaN,1615.0,NaN,3258.00,NaN,2453.00,NaN,NaN
4,NE,2000-01-01,301.13,NaN,39.05,1820.46,NaN,0.0,NaN,1670.0,NaN,3527.05,NaN,321.94,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1222969,S,2020-12-31,327.90,269.50,NaN,76.76,67.0,0.0,0.0,67.0,0.0,24.38,NaN,10.01,-0.170,0.9352
1222970,S,2020-12-31,264.23,211.00,NaN,329.22,199.0,85.0,0.0,284.0,0.0,460.67,NaN,62.66,-2.618,7.3897
1222971,S,2020-12-31,547.12,426.53,60.25,65.50,77.0,0.0,1.0,78.0,0.0,65.60,NaN,65.60,-0.005,0.1048
1222972,S,2020-12-31,154.36,132.29,NaN,82.23,77.0,0.0,0.0,77.0,0.0,81.76,NaN,81.76,-0.115,1.4416


Os dados sobre **Geração de Energia** foram adquiridos do portal de Dados Abertos do Operador Nacional do Sistema Elétrico (ONS), acessível através do seguinte link: https://dados.ons.org.br/dataset/geracao-usina-2

In [3]:
# Criar listas de anos e números
anos = list(range(2000, 2021)) # Para o desafio, deve-se considerar o range até 2021.

# Criar função para obter os dados de um ano específico, considerando apenas ano
def download_csv_ano(ano):
    try:
      if ano <= 2021:
        url = f"https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/geracao_usina_2_ho/GERACAO_USINA_{ano}.csv"
        # Imprimir a URL antes de fazer o download
        print(f"URL: {url}")
        return pd.read_csv(url, sep=';')
    except Exception as e:
        print(f"Erro ao baixar dados de {url}: {e}")
        return None

# Baixar os DataFrames com apenas no ano (correspodem os dados de 2000 a 2021)
dado_df_ano = pd.concat([download_csv_ano(ano) for ano in anos], ignore_index=True)

URL: https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/geracao_usina_2_ho/GERACAO_USINA_2000.csv
URL: https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/geracao_usina_2_ho/GERACAO_USINA_2001.csv
URL: https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/geracao_usina_2_ho/GERACAO_USINA_2002.csv
URL: https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/geracao_usina_2_ho/GERACAO_USINA_2003.csv
URL: https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/geracao_usina_2_ho/GERACAO_USINA_2004.csv
URL: https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/geracao_usina_2_ho/GERACAO_USINA_2005.csv
URL: https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/geracao_usina_2_ho/GERACAO_USINA_2006.csv
URL: https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/geracao_usina_2_ho/GERACAO_USINA_2007.csv
URL: https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/geracao_usina_2_ho/GERACAO_USINA_2008.csv
URL: https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/geracao_usina_2_ho/GER

In [ ]:
# Visualizando o dataframe
dado_df_ano.head()

,din_instante,id_subsistema,nom_subsistema,id_estado,nom_estado,cod_modalidadeoperacao,nom_tipousina,nom_tipocombustivel,nom_usina,ceg,val_geracao,id_ons
0,2000-01-01 00:00:00,N,NORTE,PA,PARA,TIPO I,HIDROELÉTRICA,Hidráulica,Tucuruí,UHE.PH.PA.002889-4.01,2422.5,NaN
1,2000-01-01 00:00:00,NE,NORDESTE,AL,ALAGOAS,TIPO I,HIDROELÉTRICA,Hidráulica,Xingó,UHE.PH.SE.027053-9.01,1995.6,NaN
2,2000-01-01 00:00:00,NE,NORDESTE,BA,BAHIA,Pequenas Usinas (Tipo III),HIDROELÉTRICA,Hidráulica,Pequenas Centrais Hidroelétricas da Chesf,-,13.0,NaN
3,2000-01-01 00:00:00,NE,NORDESTE,BA,BAHIA,TIPO I,HIDROELÉTRICA,Hidráulica,Apolônio Sales,UHE.PH.AL.001510-5.01,202.5,NaN
4,2000-01-01 00:00:00,NE,NORDESTE,BA,BAHIA,TIPO I,HIDROELÉTRICA,Hidráulica,Paulo Afonso II,UHE.PH.BA.027048-2.01,269.8,NaN


# Análise dos dados

In [4]:
# Eliminar colunas desnecessárias
dado_df_ano.drop(columns=['nom_subsistema','id_estado','nom_estado','cod_modalidadeoperacao','nom_tipocombustivel','nom_usina','ceg','id_ons'], inplace=True)
dado_hidrologico.drop(columns = ['nom_subsistema','tip_reservatorio','nom_bacia', 'nom_ree','nom_reservatorio','num_ordemcs','cod_usina'], inplace=True)

In [ ]:
# Verificar oS tipos de dados em cada coluna do dataframe dado_df
dado_df_ano.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54335427 entries, 0 to 54335426
Data columns (total 4 columns):
 #   Column         Dtype  
---  ------         -----  
 0   din_instante   object 
 1   id_subsistema  object 
 2   nom_tipousina  object 
 3   val_geracao    float64
dtypes: float64(1), object(3)
memory usage: 1.6+ GB


In [ ]:
# Verificar oS tipos de dados em cada coluna do dataframe dado_hidrologico
dado_hidrologico.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1222974 entries, 0 to 1222973
Data columns (total 16 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   id_subsistema               1222974 non-null  object 
 1   din_instante                1222974 non-null  object 
 2   val_nivelmontante           991388 non-null   float64
 3   val_niveljusante            767276 non-null   float64
 4   val_volumeutilcon           473318 non-null   float64
 5   val_vazaoafluente           1084330 non-null  float64
 6   val_vazaoturbinada          748759 non-null   float64
 7   val_vazaovertida            1000553 non-null  float64
 8   val_vazaooutrasestruturas   162117 non-null   float64
 9   val_vazaodefluente          1069790 non-null  float64
 10  val_vazaotransferida        338552 non-null   float64
 11  val_vazaonatural            1190109 non-null  float64
 12  val_vazaoartificial         156175 non-null   float64
 1

In [ ]:
# Verificar se há valores ausentes no dataframe dado_df
dado_df_ano.isna().sum()

din_instante          0
id_subsistema         0
nom_tipousina         0
val_geracao      164361
dtype: int64

In [ ]:
# Verificar se há valores ausentes no dataframe dado_hidrologico
dado_hidrologico.isna().sum()

id_subsistema                       0
din_instante                        0
val_nivelmontante              231586
val_niveljusante               455698
val_volumeutilcon              749656
val_vazaoafluente              138644
val_vazaoturbinada             474215
val_vazaovertida               222421
val_vazaooutrasestruturas     1060857
val_vazaodefluente             153184
val_vazaotransferida           884422
val_vazaonatural                32865
val_vazaoartificial           1066799
val_vazaoincremental           226103
val_vazaoevaporacaoliquida     573968
val_vazaousoconsuntivo         650335
dtype: int64

In [5]:
# Eliminar os NaNs do dataframe dado_df_ano
dado_df_ano.dropna(inplace=True)

# Inserindo zero no dataframe dado_hidrologico
dado_hidrologico.fillna(0, inplace=True)

In [7]:
# Converter a coluna 'din_instante' para datetime
dado_df_ano['din_instante'] = pd.to_datetime(dado_df_ano['din_instante'])
dado_hidrologico['din_instante'] = pd.to_datetime(dado_hidrologico['din_instante'])

In [8]:
# Agregar os dados por ano, região, estado e tipo de usina (dataframe dado_df_ano)
dado_df = dado_df_ano.assign(
    ano = dado_df_ano['din_instante'].dt.to_period('Y'),  # Cria uma nova coluna 'ano'
).groupby(['ano', 'id_subsistema', 'nom_tipousina']).agg( # Agrupa por ano, região, estado e tipo de geração
    {'val_geracao': 'sum'}  # soma a geração de energia
).reset_index()  # Reinicia o índice do DataFrame resultante

In [9]:
# Agregar os dados por ano, região, e etc (dataframe dado_hidrologico)
dado_hidrologico = dado_hidrologico.assign(
    ano=dado_hidrologico['din_instante'].dt.to_period('Y')
).groupby(['ano', 'id_subsistema']).agg(
    {'val_vazaoafluente': 'sum',
     'val_vazaoturbinada': 'sum',
     'val_vazaovertida': 'sum',
     'val_vazaooutrasestruturas': 'sum',
     'val_vazaodefluente': 'sum',
     'val_vazaotransferida': 'sum',
     'val_vazaonatural': 'sum',
     'val_vazaoartificial': 'sum',
     'val_vazaoincremental': 'sum',
     'val_vazaoevaporacaoliquida': 'sum',
     'val_vazaousoconsuntivo': 'sum'}
).reset_index()

In [ ]:
# Imprimir o resumo da estatística descritiva (VISÃO GERAL) (GERAÇÃO DE ENERGIA)
print('O resumo da estatística descritiva (VISÃO GERAL):')
print(dado_df['val_geracao'].describe())

O resumo da estatística descritiva (VISÃO GERAL):
count    2.260000e+02
mean     4.311923e+07
std      6.858393e+07
min      0.000000e+00
25%      6.548266e+06
50%      1.598755e+07
75%      4.514240e+07
max      2.923040e+08
Name: val_geracao, dtype: float64


In [ ]:
# Imprimir o resumo da estatística descritiva (VISÃO GERAL) (HIDROLÓGICOS)
print('O resumo da estatística descritiva (VISÃO GERAL):')
print(dado_hidrologico.describe())

O resumo da estatística descritiva (VISÃO GERAL):
       val_vazaoafluente  val_vazaoturbinada  val_vazaovertida  \
count       8.400000e+01        8.400000e+01      8.400000e+01   
mean        1.103516e+07        6.266168e+06      2.012057e+06   
std         1.158475e+07        8.905124e+06      2.837017e+06   
min         1.273251e+06        2.043612e+04      6.327280e+03   
25%         3.795085e+06        9.729617e+05      3.435698e+05   
50%         5.514371e+06        2.683073e+06      1.244831e+06   
75%         1.160262e+07        4.659206e+06      2.368818e+06   
max         3.842279e+07        3.186166e+07      1.588336e+07   

       val_vazaooutrasestruturas  val_vazaodefluente  val_vazaotransferida  \
count                  84.000000        8.400000e+01          8.400000e+01   
mean                10370.841667        1.094720e+07          9.023529e+04   
std                 33199.422651        1.161745e+07          2.422837e+05   
min                     0.000000        1.0

In [10]:
# Filtrar para obter apenas o tipo hidrelétrica
hidreletrica = dado_df[dado_df['nom_tipousina']=='HIDROELÉTRICA']

# Concatenando com o dataframe dado_hidrologico
hidreletrica = pd.merge(hidreletrica,dado_hidrologico, on=['id_subsistema','ano'], how='inner')

In [24]:
print('A correlação entre a geração de energia Hidroelétrica e as variáveis:\n', hidreletrica.corr().iloc[0])

A correlação entre a geração de energia Hidroelétrica e as variáveis:
 val_geracao                   1.000000
val_vazaoafluente             0.961781
val_vazaoturbinada            0.812438
val_vazaovertida              0.647832
val_vazaooutrasestruturas     0.399201
val_vazaodefluente            0.958851
val_vazaotransferida         -0.084500
val_vazaonatural              0.963768
val_vazaoartificial           0.968055
val_vazaoincremental          0.807334
val_vazaoevaporacaoliquida    0.841605
val_vazaousoconsuntivo        0.841340
Name: val_geracao, dtype: float64


<ipython-input-24-59aa695df182>:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



In [12]:
# Imprimir o resumo da estatística descritiva (POR REGIÃO) do datagrame dado_df

# Imprimindo as estatísticas descritivas de cada região
print("Resumo da região Norte (N):")
print(dado_df[dado_df['id_subsistema']=='N']['val_geracao'].describe())
print('')

print("Resumo da região Nordeste (NE):")
print(dado_df[dado_df['id_subsistema']=='NE']['val_geracao'].describe())
print('')

print("Resumo da região Sul (S):")
print(dado_df[dado_df['id_subsistema']=='S']['val_geracao'].describe())
print('')

print("Resumo da região Sudeste (SE):")
print(dado_df[dado_df['id_subsistema']=='SE']['val_geracao'].describe())

Resumo da região Norte (N):
count    3.600000e+01
mean     2.725836e+07
std      1.890360e+07
min      9.189000e+02
25%      1.567930e+07
50%      2.784750e+07
75%      4.164695e+07
max      6.719583e+07
Name: val_geracao, dtype: float64

Resumo da região Nordeste (NE):
count    6.300000e+01
mean     2.031357e+07
std      1.911823e+07
min      7.442904e+03
25%      1.875230e+06
50%      1.615676e+07
75%      3.717794e+07
max      5.908158e+07
Name: val_geracao, dtype: float64

Resumo da região Sul (S):
count    5.700000e+01
mean     2.675005e+07
std      2.914871e+07
min      1.468574e+05
25%      5.829124e+06
50%      1.001481e+07
75%      5.126855e+07
max      9.094051e+07
Name: val_geracao, dtype: float64

Resumo da região Sudeste (SE):
count    7.000000e+01
mean     8.513053e+07
std      1.073363e+08
min      0.000000e+00
25%      1.028641e+07
50%      1.570257e+07
75%      2.234524e+08
max      2.923040e+08
Name: val_geracao, dtype: float64


In [ ]:
# Calcular a matriz de correlação entre as regiões
matriz_correlacao = dado_df.pivot_table(index='ano', columns='id_subsistema', values='val_geracao', aggfunc='sum').corr()

# Exibir a matriz de correlação
print("Matriz de Correlação entre as Regiões:")
print(matriz_correlacao)

Matriz de Correlação entre as Regiões:
id_subsistema         N        NE         S        SE
id_subsistema                                        
N              1.000000  0.928667  0.534493  0.688687
NE             0.928667  1.000000  0.408512  0.729709
S              0.534493  0.408512  1.000000  0.521772
SE             0.688687  0.729709  0.521772  1.000000


In [ ]:
# Calcular a matriz de correlação entre as fontes de geração
matriz_correlacao = dado_df.pivot_table(index='ano', columns='nom_tipousina', values='val_geracao', aggfunc='sum').corr()

# Exibir a matriz de correlação
print("Matriz de Correlação entre as Fontes de geração:")
print(matriz_correlacao['HIDROELÉTRICA'])

Matriz de Correlação entre as Fontes de geração:
nom_tipousina
BOMBEAMENTO      0.807435
EOLIELÉTRICA    -0.228673
FOTOVOLTAICA     0.750684
HIDROELÉTRICA    1.000000
NUCLEAR          0.461320
TÉRMICA          0.342034
Name: HIDROELÉTRICA, dtype: float64


In [ ]:
# Visualizar graficamente os dados (VISÃO GERAL)

# Agrupar por região para facilitar a visualização do gráfico
plot_fonte = dado_df.groupby(['ano', 'nom_tipousina'])['val_geracao'].sum().reset_index()

# Converter a columa mes_ano em strings
plot_fonte['ano'] = plot_fonte['ano'].astype(str)

# Plotar o gráfico
fig = px.bar(plot_fonte, x='ano', y='val_geracao', color='nom_tipousina',
             title='Evolução da Geração de Energia por Fonte de Geração (Visão Geral)',
             labels={'val_geracao': 'Geração de Energia (MW med)', 'mes_ano': 'Data'},
             color_discrete_map={'EOLIELÉTRICA': 'blue', 'NUCLEAR': 'black', 'TÉRMICA': 'orange', 'HIDROELÉTRICA': 'purple','BOMBEAMENTO':'red', 'FOTOVOLTAICA':'brown'})

fig.update_xaxes(title_text='Data')
fig.update_yaxes(title_text='Geração de Energia (MW med)')
fig.update_layout(legend_title_text='Regiões')
fig.show()

In [13]:
# Visualizar graficamente os dados das hidrelétrica por região (VISÃO GERAL)

# Agrupar por região para facilitar a visualização do gráfico
df_plot_hidreletrica = dado_df[dado_df['nom_tipousina']=='HIDROELÉTRICA']

# Converter a columa mes_ano em strings
df_plot_hidreletrica['ano'] = df_plot_hidreletrica['ano'].astype(str)

# Plotar o gráfico
fig = px.bar(df_plot_hidreletrica, x='ano', y='val_geracao', color='id_subsistema',
             title='Evolução da Geração de Energia Hídrica por Região (Visão Geral)',
             labels={'val_geracao': 'Geração de Energia (MW med)', 'mes_ano': 'Data'},
             color_discrete_map={'N': 'blue', 'NE': 'black', 'S': 'orange', 'SE': 'purple'})

fig.update_xaxes(title_text='Data')
fig.update_yaxes(title_text='Geração de Energia (MW med)')
fig.update_layout(legend_title_text='Regiões')
fig.show()

<ipython-input-13-34dd4e994403>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_plot_hidreletrica['ano'] = df_plot_hidreletrica['ano'].astype(str)


In [20]:
# Criar Função para plotar o gráfico por Região

# Agrupar os dados por tipo de geração e
plot_por_regiao = dado_df.groupby(['ano', 'nom_tipousina','id_subsistema'])['val_geracao'].sum().reset_index()

# Convertendo a coluna 'mes_ano' para string
plot_por_regiao['ano'] = plot_por_regiao['ano'].astype(str)

# Criar função para plotar gráfico
def plotar_grafico(nome, sigla):
    fig = px.bar(plot_por_regiao[(plot_por_regiao['id_subsistema'] == f'{sigla}') & (plot_por_regiao['nom_tipousina'] == 'HIDROELÉTRICA')], x='ano', y='val_geracao', color='id_subsistema',
                 title=f'Evolução da Geração de Energia Hidrelétrica na Região {nome.upper()}',
                 labels={'val_geracao': 'Geração de Energia (MW med)', 'ano': 'Data'},
                 color_discrete_map={'N': 'blue', 'NE': 'black', 'S': 'orange', 'SE': 'purple'})

    # Calculando a média da região
    media_regiao = plot_por_regiao[plot_por_regiao['id_subsistema'] == f'{sigla}']['val_geracao'].mean()

    # Adicionando a média como uma linha horizontal
    fig.add_hline(y=media_regiao, line_color="red", annotation_text=f'Média: {media_regiao:.2f} MW med', annotation_position="bottom right")

    fig.update_xaxes(title_text='Data')
    fig.update_yaxes(title_text='Geração de Energia (MW med)')
    fig.update_layout(legend_title_text='Formas de geração')
    fig.show()

In [21]:
# Plotando o gráfico da região Nordeste
plotar_grafico('nordeste', 'NE')

In [23]:
# Plotando o gráfico da região Norte
plotar_grafico('norte', 'N')

In [ ]:
# Plotando o gráfico da região Sudeste
plotar_grafico('sudeste', 'SE')

In [ ]:
# Plotando o gráfico da região Sul
plotar_grafico('sul', 'S')

# Modelos Preditivos

## Regressão Linear Múltipla

In [27]:
hidreletrica

,ano,id_subsistema,nom_tipousina,val_geracao,val_vazaoafluente,val_vazaoturbinada,val_vazaovertida,val_vazaooutrasestruturas,val_vazaodefluente,val_vazaotransferida,val_vazaonatural,val_vazaoartificial,val_vazaoincremental,val_vazaoevaporacaoliquida,val_vazaousoconsuntivo
0,2000,N,HIDROELÉTRICA,2.743534e+07,7.396060e+06,2610565.82,3122248.85,0.00,7404885.95,0.000,1.134565e+07,7.523600e+04,4.753881e+06,0.000,0.0000
1,2000,NE,HIDROELÉTRICA,4.992822e+07,3.409218e+06,155909.00,606629.00,0.00,3434599.73,0.000,5.778557e+06,0.000000e+00,7.737677e+05,0.000,0.0000
2,2000,S,HIDROELÉTRICA,3.388110e+07,4.234219e+06,1702261.34,495100.38,0.00,4092254.05,0.000,4.738564e+06,3.432403e+05,8.670757e+05,0.000,0.0000
3,2000,SE,HIDROELÉTRICA,2.243472e+08,2.451155e+07,17823891.94,2122098.05,0.00,24172552.45,4152.780,4.093871e+07,1.125583e+07,5.700898e+06,0.000,0.0000
4,2001,N,HIDROELÉTRICA,2.817160e+07,5.353196e+06,2456390.94,1757351.85,0.00,5434280.82,0.000,8.343372e+06,6.395400e+04,3.406414e+06,40.500,58.9196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,2019,SE,HIDROELÉTRICA,2.553023e+08,3.651086e+07,31444500.33,4917632.59,157825.18,36577551.91,55736.869,3.696692e+07,8.626797e+06,1.295937e+07,32754.398,308844.1173
80,2020,N,HIDROELÉTRICA,6.719583e+07,9.256949e+06,6273686.00,2946079.00,5989.00,9225005.00,1416039.450,9.443746e+06,2.832389e+06,6.844125e+06,5995.094,30874.8736
81,2020,NE,HIDROELÉTRICA,3.751545e+07,2.839886e+06,2266240.61,37484.91,0.00,2303725.49,473327.000,4.502621e+06,0.000000e+00,6.819357e+05,3092.450,2206.8906
82,2020,S,HIDROELÉTRICA,3.937732e+07,2.632499e+06,2229035.18,351681.00,37702.07,2622721.91,1580.490,2.653718e+06,1.600520e+05,8.177896e+05,5589.027,21248.2737


In [28]:
# Importando as bibliotecas necessárias
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Elimina a coluna nom_tipousina
hidreletrica.drop(columns='nom_tipousina', inplace=True)

# Converter a coluna em ano
hidreletrica['ano'] = hidreletrica['ano'].dt.year

# Criando variáveis independentes (X) e dependente (y)
X = hidreletrica[~hidreletrica['val_geracao'].isnull()]
y = hidreletrica['val_geracao']

# Converter coluna categórica em variáveis dummy
X_dummy = pd.get_dummies(X, columns=['id_subsistema'])

# Dividir os dados em conjunto de treinamento e teste após a codificação
X_train, X_test, y_train, y_test = train_test_split(X_dummy, y, test_size=0.2, random_state=42)

# Treinar o modelo de regressão linear
regressao_linear = LinearRegression()
regressao_linear.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred_linear = regressao_linear.predict(X_test)

# Avaliar o modelo
lin_mse = mean_squared_error(y_test, y_pred_linear)
print("Mean Squared Error:", np.sqrt(lin_mse))

Mean Squared Error: 2.252002465206666e-08


## Random Forest

In [29]:
# Importando a biblioteca necessária
from sklearn.ensemble import RandomForestRegressor

# Treinar o modelo Random Forest
random_forest = RandomForestRegressor(n_estimators=100, random_state=42)
random_forest.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred_forest = random_forest.predict(X_test)

# Avaliar o modelo
forest_mse = mean_squared_error(y_test, y_pred_forest)
print("Mean Squared Error:", np.sqrt(forest_mse))

Mean Squared Error: 3517183.0638377564


## Decision Tree

In [30]:
# Importando a biblioteca necessária
from sklearn.tree import DecisionTreeRegressor

# Criar e treinar o modelo MLPRegressor
decision_tree = DecisionTreeRegressor()
decision_tree.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred_decision = decision_tree.predict(X_test)

# Avaliar o modelo
tree_mse = mean_squared_error(y_test, y_pred_decision)
print("Mean Squared Error:", np.sqrt(tree_mse))

Mean Squared Error: 7284903.203318432


## Rede Neural Artificial


In [31]:
# Importando a biblioteca necessária
from sklearn.neural_network import MLPRegressor

# Criando e treinando o modelo MLPRegressor
mlp_regressor = MLPRegressor()
mlp_regressor.fit(X_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred_mlp = mlp_regressor.predict(X_test)

# Avaliando o modelo
mlp_mse = mean_squared_error(y_test, y_pred_mlp)
print("Mean Squared Error:", np.sqrt(mlp_mse))

Mean Squared Error: 5800716.816096007


## XGBoost

In [32]:
# Importando a biblioteca necessária
from xgboost import XGBRegressor

# Treinar o modelo XGBoost
xgboost_model = XGBRegressor(n_estimators=100, random_state=42)
xgboost_model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred_xgboost = xgboost_model.predict(X_test)

# Avaliar o modelo
xgboost_mse = mean_squared_error(y_test, y_pred_xgboost)
print("Mean Squared Error:", np.sqrt(xgboost_mse))


Mean Squared Error: 2322597.8093350637


**Comentário**:

Ao analisar e comparar os modelos com base em suas métricas de Erro Quadrático Médio (EQM), é evidente que o modelo de Regressão Linear obteve um excelente resultado.

No entanto, é fundamental destacar a importância da validação cruzada na avaliação do desempenho dos modelos de forma mais robusta. A validação cruzada desempenha um papel crucial na estimativa da capacidade de generalização do modelo para dados não observados. Portanto, oferece uma avaliação mais confiável de sua performance, garantindo que o modelo seja capaz de fazer previsões precisas em diferentes conjuntos de dados.



## Cross- Validation

In [33]:
# Importando a biblioteca necessária
from sklearn.model_selection import cross_val_score

# Regressão Linear

# Calculando os scores de validação cruzada usando a Regressão Linear
scores = cross_val_score(regressao_linear, X_train, y_train,
                         scoring="neg_mean_squared_error", cv=10)

# Calculando a raiz quadrada negativa do erro quadrático médio para cada fold
lin_rmse_scores = np.sqrt(-scores)

# Definindo uma função para exibir os scores
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

# Exibindo os scores de validação cruzada
display_scores(lin_rmse_scores)

Scores: [1.47002375e-08 4.11953298e-08 5.18490670e-08 4.95067314e-08
 5.52729216e-08 1.31904438e-07 2.36448006e-08 3.91893929e-08
 4.63295929e-08 4.40782292e-08]
Mean: 4.976707411580434e-08
Standard deviation: 2.9871158323481506e-08


In [34]:
# Random Forest

# Calculando os scores de validação cruzada usando a Floresta Aleatória
scores = cross_val_score(random_forest, X_train, y_train,
                         scoring="neg_mean_squared_error", cv=10)

# Calculando a raiz quadrada negativa do erro quadrático médio para cada fold
forest_rmse_scores = np.sqrt(-scores)

# Exibindo os scores de validação cruzada
display_scores(forest_rmse_scores)

Scores: [ 7872211.18973469  1426200.77682544 14515466.61167487  5102333.58774786
  3531152.92311207  4580566.83747063  1600377.49385593 21244893.40752769
  2395605.49512387 12649308.2381828 ]
Mean: 7491811.656125585
Standard deviation: 6265518.174023096


In [35]:
# Decision Tree

# Calculando os scores de validação cruzada usando a árvore de decisão
scores = cross_val_score(decision_tree, X_train, y_train,
                         scoring="neg_mean_squared_error", cv=10)

# Calculando a raiz quadrada negativa do erro quadrático médio para cada fold
tree_rmse_scores = np.sqrt(-scores)

# Exibindo os scores de validação cruzada
display_scores(tree_rmse_scores)

Scores: [11014351.20948656  2183645.66014593  9793827.92165396  4261759.85091003
  5459954.26324018  2511376.62188214  2484887.99431519 39992180.54392149
  4955806.78122083 11985448.79232212]
Mean: 9464323.963909842
Standard deviation: 10743881.570848094


In [38]:
# Rede Neural

# Instanciando o modelo
mlp_regressor = MLPRegressor(max_iter=1000) #Deve-se alterar o valor de iteração para não se limitar a 200 iterações (padrão do modelo)

# Calculando os scores de validação cruzada usando a árvore de decisão
scores = cross_val_score(mlp_regressor, X_train, y_train,
                         scoring="neg_mean_squared_error", cv=10)

# Calculando a raiz quadrada negativa do erro quadrático médio para cada fold
neural_rmse_scores = np.sqrt(-scores)

# Exibindo os scores de validação cruzada
display_scores(neural_rmse_scores)

Scores: [ 4551517.01928419  5519049.82514115  8126115.74608717    71951.9817998
  7095608.75398031 11582792.66828947  2863269.17683769 14963901.35457524
    71234.4079934   5788381.69160417]
Mean: 6063382.2625592565
Standard deviation: 4468835.960289245


In [39]:
# XGBoost

# Criando o modelo XGBoost
xgboost_model = XGBRegressor(n_estimators=100, random_state=42)

# Calculando os scores de validação cruzada usando XGBoost
scores = cross_val_score(xgboost_model, X_train, y_train,
                         scoring="neg_mean_squared_error", cv=10)

# Calculando a raiz quadrada negativa do erro quadrático médio para cada fold
xgboost_rmse_scores = np.sqrt(-scores)

# Exibindo os scores de validação cruzada
display_scores(xgboost_rmse_scores)

Scores: [10278104.85738858  1473474.59896124 30676760.73521722 20837411.62607789
  3726382.22697289  5990589.61886923  2504862.37849964 12025996.52426917
  5296328.61498979 11792550.83103181]
Mean: 10460246.201227743
Standard deviation: 8698803.376046753


**Resultado**: O modelo de Regressão Linear destacou-se com um desempenho excepcional em comparação com os demais modelos avaliados.

In [40]:
# Cria uma figura contendo dois gráficos de linhas: um para os valores reais e outro para as previsões do modelo final
fig = go.Figure(data=[go.Scatter(y=y_test, name='Real'),
                      go.Scatter(y=y_pred_linear, name='Previsões')])

# Atualiza os nomes dos eixos x e y
fig.update_xaxes(title_text='Amostras')
fig.update_yaxes(title_text='Valor')

fig.show()